In [28]:
import pandas as pd
import os

<b>First Clone Repo from "https://github.com/CSSEGISandData/COVID-19.git"</b>
<br>
<b>To Get All Covid-19 Recorded Data</b>

In [29]:
DIR_PATH = './csse_covid_19_daily_reports/'
files = os.listdir(DIR_PATH)

In [30]:
# Filter unneeded files from direcctory
validFileNames = []
for fileName in files:
    name, extention = fileName.split('.')
    if extention == 'csv':
        validFileNames.append(fileName)

# print first file for validation
print(validFileNames[0])

01-01-2021.csv


In [31]:
# Explore only one file to have insights about the data
csv_path = DIR_PATH + validFileNames[0]
df = pd.read_csv(csv_path)
df.head(5)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-01-02 05:22:33,33.93911,67.709953,52513,2201,41727,0.0,Afghanistan,0.000000,4.252222
1,NaN,NaN,NaN,Albania,2021-01-02 05:22:33,41.15330,20.168300,58316,1181,33634,23501.0,Albania,2026.409062,2.025173
2,NaN,NaN,NaN,Algeria,2021-01-02 05:22:33,28.03390,1.659600,99897,2762,67395,29740.0,Algeria,227.809861,2.764848
3,NaN,NaN,NaN,Andorra,2021-01-02 05:22:33,42.50630,1.521800,8117,84,7463,570.0,Andorra,10505.403482,1.034865
4,NaN,NaN,NaN,Angola,2021-01-02 05:22:33,-11.20270,17.873900,17568,405,11146,6017.0,Angola,53.452981,2.305328


In [32]:
# filter egypt only data
egypt_filtered = df[df['Country_Region'] == 'Egypt']
egypt_filtered.head(5)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
195,NaN,NaN,NaN,Egypt,2021-01-02 05:22:33,26.820553,30.802498,139471,7687,112826,18958.0,Egypt,136.289455,5.51154


In [33]:
# print the format I want to have to confirm befoe running on all files
tp = [egypt_filtered.iloc[0]['Confirmed'], egypt_filtered.iloc[0]['Deaths'], egypt_filtered.iloc[0]['Recovered'], egypt_filtered.iloc[0]['Active']]
print(tp)

[139471, 7687, 112826, 18958.0]


In [34]:
# filter All the files for Egypt Only
filter_egypt_data = []

for fileName in validFileNames:
    csv_path = DIR_PATH + fileName
    df = pd.read_csv(csv_path)
    #print("Reading {}".format(fileName))

    noEgypt = False
    firstPhase = False


    try:
        egypt_filtered = df[df['Country_Region'] == 'Egypt']
    except:
        firstPhase = True

    if firstPhase:  
        try:
            egypt_filtered = df[df['Country/Region'] == 'Egypt']
        except:
            noEgypt = True

    if egypt_filtered['Confirmed'].count() > 0 and noEgypt == False:
        name, extention = fileName.split('.')
        month, day, year = name.split('-')

        timestamp = month + '/' + day + '/' + year
        tp = [timestamp, egypt_filtered.iloc[0]['Confirmed'], egypt_filtered.iloc[0]['Deaths'], egypt_filtered.iloc[0]['Recovered']]
        filter_egypt_data.append(tp)

# print one sample to validate
print(filter_egypt_data[0])

['01/01/2021', 139471, 7687, 112826]


In [35]:
# create a dataframe from the list and add it's header
df = pd.DataFrame(filter_egypt_data)
csv_header = ['timestamp', 'confirmed', 'deaths', 'recovered']
df.columns = csv_header

<h3>Explore the new filtered data</h3>

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 634 entries, 0 to 633
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   timestamp  634 non-null    object 
 1   confirmed  634 non-null    int64  
 2   deaths     634 non-null    float64
 3   recovered  538 non-null    float64
dtypes: float64(2), int64(1), object(1)
memory usage: 19.9+ KB


In [37]:
df['timestamp']= pd.to_datetime(df['timestamp'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 634 entries, 0 to 633
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   timestamp  634 non-null    datetime64[ns]
 1   confirmed  634 non-null    int64         
 2   deaths     634 non-null    float64       
 3   recovered  538 non-null    float64       
dtypes: datetime64[ns](1), float64(2), int64(1)
memory usage: 19.9 KB


In [38]:
df.head(5)

,timestamp,confirmed,deaths,recovered
0,2021-01-01,139471,7687.0,112826.0
1,2021-01-02,140878,7741.0,113480.0
2,2021-01-03,142187,7805.0,113898.0
3,2021-01-04,143464,7863.0,114601.0
4,2021-01-05,144583,7918.0,115414.0


In [39]:
df.describe(include='all', datetime_is_numeric=True)

,timestamp,confirmed,deaths,recovered
count,634,634.000000,634.000000,538.000000
mean,2020-12-26 12:00:00,154495.113565,8784.823344,96011.682156
min,2020-02-14 00:00:00,1.000000,0.000000,0.000000
25%,2020-07-21 06:00:00,89244.750000,4409.250000,17239.750000
50%,2020-12-26 12:00:00,131928.000000,7378.500000,100390.500000
75%,2021-06-02 18:00:00,265256.000000,15211.000000,150799.000000
max,2021-11-08 00:00:00,338414.000000,19130.000000,232179.000000
std,NaN,104262.368974,6093.516938,72216.123917


In [40]:
# sort Data by date
df.sort_values('timestamp', inplace=True)

In [41]:
df.head(5)

,timestamp,confirmed,deaths,recovered
44,2020-02-14,1,0.0,0.0
46,2020-02-15,1,0.0,0.0
48,2020-02-16,1,0.0,0.0
50,2020-02-17,1,0.0,0.0
52,2020-02-18,1,0.0,0.0


In [42]:
# calculate Active cases
df['active'] = df['confirmed'] - df['deaths'] - df['recovered']

In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 634 entries, 44 to 580
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   timestamp  634 non-null    datetime64[ns]
 1   confirmed  634 non-null    int64         
 2   deaths     634 non-null    float64       
 3   recovered  538 non-null    float64       
 4   active     538 non-null    float64       
dtypes: datetime64[ns](1), float64(3), int64(1)
memory usage: 29.7 KB


In [44]:
df['confirmed'][538:]

390    284523
392    284580
394    284641
396    284706
398    284789
        ...  
572    334751
574    335673
576    336582
578    337485
580    338414
Name: confirmed, Length: 96, dtype: int64

In [45]:
df.describe(include='all', datetime_is_numeric=True)

,timestamp,confirmed,deaths,recovered,active
count,634,634.000000,634.000000,538.000000,538.000000
mean,2020-12-26 12:00:00,154495.113565,8784.823344,96011.682156,24671.804833
min,2020-02-14 00:00:00,1.000000,0.000000,0.000000,0.000000
25%,2020-07-21 06:00:00,89244.750000,4409.250000,17239.750000,4053.500000
50%,2020-12-26 12:00:00,131928.000000,7378.500000,100390.500000,26036.000000
75%,2021-06-02 18:00:00,265256.000000,15211.000000,150799.000000,41171.250000
max,2021-11-08 00:00:00,338414.000000,19130.000000,232179.000000,56636.000000
std,NaN,104262.368974,6093.516938,72216.123917,19377.159803


In [46]:
df.head(5)

,timestamp,confirmed,deaths,recovered,active
44,2020-02-14,1,0.0,0.0,1.0
46,2020-02-15,1,0.0,0.0,1.0
48,2020-02-16,1,0.0,0.0,1.0
50,2020-02-17,1,0.0,0.0,1.0
52,2020-02-18,1,0.0,0.0,1.0


In [48]:
df.head(5)

,timestamp,confirmed,deaths,recovered,active
44,2020-02-14,1,0.0,0.0,1.0
46,2020-02-15,1,0.0,0.0,1.0
48,2020-02-16,1,0.0,0.0,1.0
50,2020-02-17,1,0.0,0.0,1.0
52,2020-02-18,1,0.0,0.0,1.0


In [49]:
df.describe(include='all',datetime_is_numeric=True)

,timestamp,confirmed,deaths,recovered,active
count,634,634.000000,634.000000,538.000000,538.000000
mean,2020-12-26 12:00:00,154495.113565,8784.823344,96011.682156,24671.804833
min,2020-02-14 00:00:00,1.000000,0.000000,0.000000,0.000000
25%,2020-07-21 06:00:00,89244.750000,4409.250000,17239.750000,4053.500000
50%,2020-12-26 12:00:00,131928.000000,7378.500000,100390.500000,26036.000000
75%,2021-06-02 18:00:00,265256.000000,15211.000000,150799.000000,41171.250000
max,2021-11-08 00:00:00,338414.000000,19130.000000,232179.000000,56636.000000
std,NaN,104262.368974,6093.516938,72216.123917,19377.159803


In [50]:
# save csv File
df.to_csv('egy_covid_cases.csv', index=False)